In [ ]:
import json
import spacy
from spacy import displacy

with open("/Users/leo/PycharmProjects/EECS4080/train-v2.0.json") as f:
    data = json.load(f)
count = 0
result = []
for i in range(2):
    for d in data["data"][i]["paragraphs"]:
        if count == 20:
            break
        if "context" in d.keys():
            print(d["context"])
            result.append(d["context"])
            print("\n---------------------------------------------------------------------------\n")
            count += 1
    
print(count)
            

In [ ]:
nlp = spacy.load("en_core_web_sm")
text="\n---------------------------------------------------------------------------------\n".join(result)
print(text)   
doc = nlp(text)
displacy.serve(doc, style="ent")


##### Obtain top 10 frequently occurred entities recognized as person
Generally, NER of the full name of a person is more accurate than the NER of a given name \
Based on this, I do the following steps: 

1. Traverse the entity list of the document, if the entity is labeled as a person, we obtain the person's full name (change to lowercase, remove stop words, remove punctuation). We add the full name to a dictionary and set the count to 1. (full name is key, count is value). For each entity, when added to the dictionary, we check if it is a substring of the keys of the dict. If so, it is a given name, we increment the count of the full name, and not add the given name to the dictionary. If it is not a substring of the keys, then it is a new name, we add it to the dictionary and set count to 1.

2. Convert the dictionary to a list, and sort it in reverse order so we can have the top10 most frequently occurred person's name.

3. Remove entities in the top10 person's name from the entity list of the document.

4. Update the label of the names to PERSON, and added them back to the entity list. 

The reason for doing this is that, we can make sure the given name and the full name are labeled consistently with PERSON. Another reason is that NER of full name is more accurate, so we want to trust full name entity labels rather than those of Given names.  



In [6]:
import string
from spacy.tokens import Span

text = "\n\n".join(result)
doc = nlp(text)
dict = {}


# for chunk in doc.noun_chunks:
#     print(f"{chunk.text:50}{chunk.root}")
    
    
for ent in doc.ents:
    entity = ""
    entity_type = ""
    for token in ent:
        # print(token.text, token.ent_type_, type(token.ent_type_), "\n")
        entity_type = token.ent_type_        
        if entity_type != "PERSON":
            continue
        if token.is_stop:  # remove stop words
            continue
        if token.text in string.punctuation:
            continue
        token = token.lemma_.lower()  # convert token to lemma and lowercase
        if token.isnumeric():  # remove numbers
            continue
        entity += token + " "
    entity = entity.strip()
    found = False
    for k in dict.keys():
        if entity in k.split():
            dict[k] += 1
            found = True
    if not found:
        dict[entity] = 1


dict_list = []
for key, value in dict.items():
    dict_list.append((key, value))

# sort the entity -> count dict to get top10 person entities        
dict_list = sorted(dict_list, key=lambda x: x[1], reverse=True)
top10 = dict_list[:10]


top10_list = []
for e in top10:
    top10_list.append(e[0])
    
    
top10_str = " ".join(top10_list)
print(top10_list)

['beyoncé giselle knowles carter', 'beyoncé giselle knowles', 'child', 'latavia roberson', 'mathew knowles', 'etta james', 'letoya luckett', "b'day", 'song', 'girl']


In [7]:
change_list = []
for ent in doc.ents:
    if ent.text.lower() in top10_str:
        change_list.append(ent)
        ents = list(doc.ents)
        ents.remove(ent)
        doc.ents = tuple(ents)
update_ent = []
print(change_list)
for e in change_list:
    span = Span(doc, e.start, e.end, label="PERSON")
    update_ent.append(span)
print(update_ent)
doc.ents = list(doc.ents) + update_ent


[Mathew Knowles, Beyoncé, Child, B'Day, Beyoncé, Etta James, Song, Beyoncé, Beyoncé, Beyoncé, Child, Beyoncé Giselle Knowles, Mathew Knowles, Beyoncé, Beyoncé, Mathew, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, LaTavia Roberson, Girl, Girl, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Knowles, Child, Beyoncé, LeToya Luckett, Roberson, Mathew, Beyoncé, Luckett, Roberson, Beyoncé, Child, Beyoncé, Beyoncé, Child, Beyoncé, Luckett, Roberson, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Child, Child, Child, Beyoncé, B'Day, Beyoncé, B'Day, Beyoncé, Beyoncé, Beyoncé, B'Day, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Etta James, James, Beyoncé, Beyoncé, James, Beyoncé, Song, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé]
[Mathew Knowles, Beyoncé, Child, B'Day, Beyoncé, Etta James, Song, Beyoncé, Beyoncé, Beyoncé, Child, Beyoncé Giselle Knowles, Mathew Knowles, Beyoncé, Beyoncé, Mathew, Beyoncé, Beyoncé, Beyoncé, Beyoncé, Beyoncé,

In [ ]:
displacy.serve(doc, style="ent")

/Users/leo/opt/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

